# docker构建镜像
----------------------------------

解决1：
Dockerfile

# This vision_insight Dockerfile
# Version 1.0

# Base images 基础镜像
FROM nvidia/cuda:10.0-runtime-centos7
# MAINTAINER 维护者信息
MAINTAINER Joshua

# RUN 执行以下命令
RUN mkdir -p /data/deploy/sv-keyphrase-tag/server/
RUN mkdir -p /data/deploy/sv-keyphrase-tag/server/resources/rbtl3/
COPY ./server /data/deploy/sv-keyphrase-tag/server/
COPY ./check.sh /
COPY ./libstdc++.so.6.0.25 /
RUN yum install gcc libffi-devel python-devel openssl-devel -y --skip-broken
RUN pip3 install --upgrade pip
RUN pip3 install -r /data/deploy/sv-keyphrase-tag/server/requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple
RUN rm -fr /usr/local/lib64/python3.6/site-packages/paddle*
RUN pip3 install paddlepaddle-gpu==2.3.0 -i https://pypi.tuna.tsinghua.edu.cn/simple
ENV LANG "en_US.UTF-8"
RUN chmod -R 777 /data/deploy/sv-keyphrase-tag/server/*
RUN chmod -R 777 /check.sh
# 启动服务
#CMD ["/bin/bash", "-c", "/data/deploy/sv-keyphrase-tag/server/start.sh"]


解决2：
Dockerfile

# This vision_insight Dockerfile
# Version 1.0

# Base images 基础镜像
FROM nvidia/cuda:10.0-runtime-centos7
# MAINTAINER 维护者信息
MAINTAINER Joshua

# RUN 执行以下命令
RUN mkdir -p /data/deploy/query_intention_server/server/
COPY ./server /data/deploy/query_intention_server/server/
COPY ./check.sh /

COPY ./libstdc++.so.6.0.25 /usr/lib64/libstdc++.so.6.0.25
RUN rm -rf /usr/lib64/libstdc++.so.6
RUN ln -s /usr/lib64/libstdc++.so.6.0.25 /usr/lib64/libstdc++.so.6


RUN yum install -y vim git less \
    && pip3 install torch==1.7.1   --extra-index-url https://download.pytorch.org/whl/cpu \
    && pip3 install -r /data/deploy/query_intention_server/server/requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple

ENV LANG "en_US.UTF-8"
RUN chmod -R 777 /data/deploy/query_intention_server/server/*

# docker编译
----------------------------------

解决：
## GPU版本
docker build -t keyphrase_tag:v0 --no-cache .

docker run -it --name=keyphrase_predict --ipc=host --net=host --privileged=true --runtime=nvidia -v /video_online/cpp_thirdparty/cuda/lib64:/usr/local/cuda/lib64 -v /video_online/python_model/new_words_tag_model/rbtl3:/data/deploy/sv-keyphrase-tag/server/resources/rbtl3 -v /video_online/log_server_online/keyphrase_tag_log:/data/deploy/sv-new-words-tag/server/logs --shm-size 16g  -p 4070:4070 -p 4071:4071 -e NVIDIA_VISIBLE_DEVICES=all -e NVIDIA_DRIVER_CAPABILITIES=compute,utility,video keyphrase_tag:v0 bash

docker start keyphrase_predict
docker exec -ti keyphrase_predict /bin/bash


## CPU版本

docker build -t query_intention:v0 --no-cache .

docker run -it --name=query_intention --ipc=host --net=host --privileged=true  -v /mnt/video_online_juicefs/python_model/text_retrieval_models:/data/deploy/models -v /mnt/video_online_juicefs/config_online/bid_config:/data/bid_config -v /mnt/video_online_juicefs/log_server_online/query_intention_server_log:/data/deploy/query_intention_server/server/log --shm-size 32g  -p 7002:7002 query_intention:v0 bash

"/bin/sh" "-c" "/data/deploy/query_intention_server/start.sh  1 data/config.yaml ; tail -f /dev/null"

# 大模型llm镜像构建

In [ ]:
docker build -t chatglm2:v0 --no-cache .
docker run --gpus all --ipc=host --ulimit memlock=-1 -v `pwd`/models:/app/models/chatglm2 -p 7860:7860 -it --rm chatglm2:v0

docker run -it --ipc=host --ulimit memlock=-1 --runtime=nvidia  -e NVIDIA_VISIBLE_DEVICES=all -e NVIDIA_DRIVER_CAPABILITIES=compute,utility,video -v `pwd`/models/chatglm2-6b:/app/models/chatglm2 -p 6868:6868 chatglm2:v0 bash